In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_9080/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_9080/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_2_8_4,0.971904,0.809068,0.868000,0.889861,0.098489,1.276766,0.472155,0.238100,0.741340,0.313830,0.941996,0.318772,70.635616,142.390396,"Hidden Size=[8], regularizer=0.02, learning_ra..."
1,model_2_8_8,0.972342,0.806178,0.853540,0.884924,0.096954,1.296088,0.523877,0.248774,0.739272,0.311374,0.942900,0.316278,70.667039,142.421819,"Hidden Size=[8], regularizer=0.02, learning_ra..."
2,model_2_38_2,0.977316,0.802883,0.780071,0.927030,0.079520,1.318124,0.786672,0.127285,0.893141,0.281993,0.953168,0.286435,71.063486,142.818266,"Hidden Size=[8], regularizer=0.02, learning_ra..."
3,model_2_37_7,0.978542,0.800847,0.792793,0.938679,0.075222,1.331740,0.741168,0.121647,0.861760,0.274267,0.955699,0.278586,71.174618,142.929398,"Hidden Size=[8], regularizer=0.02, learning_ra..."
4,model_2_9_2,0.973212,0.800400,0.842690,0.979309,0.093905,1.334725,0.562689,0.056304,0.660946,0.306439,0.944696,0.311265,70.730950,142.485730,"Hidden Size=[8], regularizer=0.02, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,model_2_2_0,0.861594,0.622002,0.722290,0.759814,0.485185,2.527674,0.993351,0.562776,1.661904,0.696552,0.714258,0.707522,67.446450,139.201229,"Hidden Size=[8], regularizer=0.02, learning_ra..."
326,model_3_1_0,0.859465,0.620543,0.713773,0.737373,0.492646,2.537429,1.023817,0.657322,1.880455,0.701888,0.400385,0.712942,99.415928,205.960904,"Hidden Size=[12], regularizer=0.02, learning_r..."
327,model_1_4_3,0.876140,0.617585,0.642222,0.876352,0.434193,2.557212,1.279750,0.522153,1.511713,0.658933,0.831339,0.669311,35.668531,72.633114,"Hidden Size=[4], regularizer=0.02, learning_ra..."
328,model_1_4_0,0.869228,0.611189,0.640223,0.870983,0.458423,2.599980,1.286901,0.544827,1.536316,0.677070,0.821927,0.687733,35.559925,72.524509,"Hidden Size=[4], regularizer=0.02, learning_ra..."
